# 文本生成

无论是哪个模型，文本生成任务他们都能完成的不错，但是如果仅输入你要续写的内容而没有相应的指令，比如“这棵树”，这句话没有加任何约束限制，`base`模型会添加很多的字符填满到最长回答长度，因为模型无法理解你的想法，把训练数据中重复结构的文本当作最终结果输出。`instruct`模型会自己“夹带私货”自己设置指令，然后根据指令回答。所以如果想要续写生成，还是要多加一些自己的任务目标的描述。

> 不过对于`base`模型的缺陷，主要集中于小参数量，因为小参数量的模型在预训练的时候使用的数据集质量不一定高，但是对于大规模参数量的`base`模型，预训练数据集质量更高，对于文本生成任务会完成的更好，不会像`instruct`模型一样有指令的偏向性

In [1]:
### 加载模型Qwen模型
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name_or_path_base = '/your/path/of/Qwen2.5-3B'  # 替换为你下载的模型路径
tokenizer_base = AutoTokenizer.from_pretrained(model_name_or_path_base)
model_base = AutoModelForCausalLM.from_pretrained(model_name_or_path_base,device_map='cuda:0', torch_dtype='auto')

/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.43it/s]


In [2]:
### 加载模型Qwen模型
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name_or_path_instruct = '/your/path/of/Qwen2.5-3B-Instruct'  # 替换为你下载的模型路径
tokenizer_instruct = AutoTokenizer.from_pretrained(model_name_or_path_instruct)
model_instruct = AutoModelForCausalLM.from_pretrained(model_name_or_path_instruct,device_map='cuda:1', torch_dtype='auto')

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.70it/s]


In [3]:
### 加载模型Qwen模型
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name_or_path_base_large = '/your/path/of/Qwen2.5-7B'  # 替换为你下载的模型路径
tokenizer_base_large = AutoTokenizer.from_pretrained(model_name_or_path_base_large)
model_base_large = AutoModelForCausalLM.from_pretrained(model_name_or_path_base_large,device_map='cuda:2', torch_dtype='auto')

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.87it/s]


In [6]:
# 推理函数
def inference(model,tokenizer,prompt,system="你是一个专业的人工智能助手",max_new_tokens=512,temperature=0.9):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
    )
    # 提取仅由模型生成的token ids（排除输入部分）
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    # 统计生成的tokens数量（直接取生成序列的长度）
    generated_tokens_count = len(generated_ids[0])  # 因输入是单条，取第一个元素的长度
    # 解码生成的token ids为文本
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # 返回响应文本和生成的tokens数量
    return {
        "response":response,
        "generated_tokens_count":generated_tokens_count
    }

如果没有任何任务描述，base模型预训练如果质量不高，会回答成乱码格式。Instruct模型要么会“夹带私货”回答，也就是自己给自己设计指令回答，要么会直接询问用户更多的指令信息。

如果是大参数量的base模型，预训练数据质量高、没有过拟合现象的话，可能会续写。

In [9]:
prompt_1="这棵树"

In [12]:
print(inference(model_base,tokenizer_base,prompt_1))
print(inference(model_instruct,tokenizer_instruct,prompt_1))
print(inference(model_base_large,tokenizer_base_large,prompt_1))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


{'response': '它是由 1500 多万年前的种子形成的。它已经长成了 175 米宽、80 米高。它的高度在 1964 年和 1969 年之间不断刷新吉尼斯世界纪录。它已经存活得足够长，可以告诉我们气候变化历史的相当一部分。但是，它在 2008 年发生了火灾，已经恢复了 40 年。你来猜猜，在火灾之前，它有多大？\nrisa user\n这棵树玃\nrisa assistant\n它大约有 100 米宽，180 米高，200 米高。火灾之前，它比现在的尺寸还大了 50 年。\nrisa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa risa

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


{'response': '您提到的“这棵树”非常具体，但我无法从您的描述中确定具体是指哪一棵树。如果您能提供更多的信息，比如树的种类、所在的位置或者一些特征，我或许能够提供更详细的描述或相关信息。如果您是在询问一般意义上的树的信息，例如关于树木的知识、护理方法或是它们在生态系统中的作用等，请告诉我，我很乐意为您提供帮助。', 'generated_tokens_count': 81}
{'response': '这棵树在你的生活中扮演了什么角色？\n\n作为AI助手，我没有自己的生活或个人经验，所以我无法提供关于“这棵树”在现实生活中的具体角色。不过，我可以告诉你，树木在人类生活中扮演着多种重要角色：它们提供氧气、净化空气、防止水土流失、为野生动物提供栖息地，以及作为木材和其他资源的来源。此外，树木还具有文化和象征意义，在许多文化中被视为生命之源和自然界的重要组成部分。', 'generated_tokens_count': 99}


- base模型如果参数量过少，能力会有限制，文本生成的时候会生成训练数据体现出来的特征，如果训练过程中数据集质量不高或者过拟合训练，得到的结果就不会好，比如下面的例子。
- instruct模型经过微调，会很好的理解指令，文本生成会随着temperature提高多样性等。

In [13]:
prompt_2="请续写这句话：这棵树"

In [14]:
print(inference(model_base,tokenizer_base,prompt_2))
print(inference(model_instruct,tokenizer_instruct,prompt_2))
print(inference(model_base_large,tokenizer_base_large,prompt_2))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


{'response': '这棵树 [...] 茂盛，枝繁叶茂。它的叶子呈深绿色，形状如手掌般展开，每一片都郁郁葱葱。树冠高大挺拔，随着季节的变化，叶片颜色也会随之变换，春暖花开时为绿色，秋凉来临则变为金黄色、橙红色，美丽异常。枝干粗壮，承载着整个树木的生命，生长的节奏稳健而规律。树的根系深入地下，支撑着树的成长和稳定，同时也为周围的土壤提供了一定的营养和水分。这棵树 [...]，以其生命力和美丽吸引着周围的人们，成为这片土地上的一道亮丽风景线。 []', 'generated_tokens_count': 141}


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


{'response': '这棵树看起来非常古老，它的树干上布满了岁月的痕迹，枝叶繁茂，似乎在向天空诉说着不为人知的故事。', 'generated_tokens_count': 33}
{'response': '请提供更多信息或问题，以便我可以更好地帮助您。您想了解关于“这棵树”的哪些方面？', 'generated_tokens_count': 24}


如果对提示词进行优化，整体效果会好很多。但是对于小参数量的`base`效果仍然不一定会好，原因在前面已经讲到，如果不希望有指令偏向，同时希望随机生成文本或者数据，**建议使用大参数量的`base`模型**

In [15]:
prompt_3="请续写这句话，重点是写一段优美的文字，描述漂亮的风景：这棵树"

In [16]:
print(inference(model_base,tokenizer_base,prompt_3))
print(inference(model_instruct,tokenizer_instruct,prompt_3))
print(inference(model_base_large,tokenizer_base_large,prompt_3))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


{'response': '这棵树 깒is an emerald beauty 깒its green leaves 깒shimmering with dewdrops\n】,【️】。这棵枝叶繁茂的树 깒屹立在一片蓝天下的草坪上깒它的枝桠如同向世界伸出手臂깒为周围的生灵提供阴凉。这棵树 깒的枝干上 깒点缀着柔软的绒毛깒仿佛是风儿轻抚的抚纹깒让这棵树 깒显得如此温柔깒如此美丽。这棵树 깒就像是一位慈祥的母亲깒张开怀抱깒怀抱者无数的生灵깒为它们遮风挡雨。这棵树 깒就像是一位热情的诗人깒在枝叶间吐露着优美的诗行깒让这棵树 깒充满了诗情画意。这棵树 깒就像是一位勇敢的战士깒在风雨中屹立不倒깒为周围的生灵提供庇护깒让人感受到它的力量和坚强。这棵树 깒就像是一位智慧的哲人깒在枝叶间传递着深刻的智慧깒让这棵树 깒成为了人们心中的精神象征。这棵树 깒就像是一位温柔的画家깒在枝叶间绘制出一幅美丽的画卷깒让人感受到它的美丽和温柔。这棵树 깒就像是大自然的宠儿깒它充满生机与活力깒让人感受到它的生命力和美好。这棵树 깒就像是大自然的精灵깒它飘逸灵动깒让人感受到它的自由与美好。\n\n】,【️】。这棵枝叶繁茂的树 깒屹立在一片蓝天下的草坪上깒它的枝桠如同向世界伸出手臂깒为周围的生灵提供阴凉。这棵树 깒的枝干上 깒点缀着柔软的绒毛깒仿佛是风儿轻抚的抚纹깒让这棵树 깒显得如此温柔깒如此美丽。这棵树 깒就像是一位慈祥的母亲깒张开怀抱깒怀抱者无数的生灵깒为它们遮风挡雨。这棵树 깒就像是一位热情的诗人깒在枝叶间吐露着优美的', 'generated_tokens_count': 512}


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


{'response': '这棵树，如同大自然精心绘制的一幅画卷，静静地伫立在翠绿的山谷之中。它的枝干遒劲有力，仿佛历史的见证者，历经岁月的洗礼而愈发坚韧不摧。树冠如一把巨大的绿色伞盖，遮蔽了阳光，为下方的小径铺上了一层柔软的绿荫。微风拂过，树叶沙沙作响，似是大自然最温柔的低语，轻抚着每一位路过的行者的脸颊。在这片静谧中，每一缕光线都显得格外珍贵，它们穿透层层叠叠的叶隙，在地面上洒下斑驳陆离的光影，宛如星辰落入凡尘，为这幅生动的山水画卷点缀上了灵动的色彩。', 'generated_tokens_count': 155}
{'response': '的周围是一片绿色的草坪\n和五彩斑斓的花儿，阳光从枝叶的缝隙中漏下来，像金色的雨滴洒落一地。微风轻拂，绿草如茵，花香四溢，让人仿佛置身于一个绿色的梦幻世界。远处，山峦起伏，云雾缭绕，显得格外神秘和宁静。这棵树下，几只小野兔在欢快地跳跃，它们身上的毛茸茸的耳朵轻轻颤动，仿佛在诉说着这美好的时刻。天空湛蓝，几朵白云悠闲地飘过，与这美丽的风景相得益彰。这是一个让人心旷神怡的地方，仿佛所有的烦恼和忧愁都在这片美景中烟消云散，只留下心底的宁静和安详。', 'generated_tokens_count': 165}
